In [56]:
import random
from collections import defaultdict
import itertools


In [81]:
def draw_board(board):
  for row in board:
    print(' '.join(map(str, row)))

def compress_board(board):
    W, H = len(board), len(board[0])
    seq = []
    for x in range(W):
        for y in range(H):
            if board[x][y] != 0:
                seq.append(board[x][y])
    jseq = ','.join(map(str, seq))
    print("Board{W: %d, H: %d, b: []int{%s}}" % (W, H, jseq))

In [28]:
"""
def make_board(w, h, seed):
    random.seed(seed)
    cnt, total = defaultdict(int), 0

    board = [[0 for _ in range(h)] for _ in range(w)]
    for x in range(w):
        for y in range(h):
            v = random.randint(1, 9)
            board[x][y] = v
            cnt[v]+=1
            total += v
  
    if total % 10 != 0: return None
    if cnt[9] > cnt[1]: return None

    return board

seed = 0
while True:
  board = make_board(3, 3, seed)
  if board is not None:
    print(seed)
    break
  seed += 1

draw_board(board)
"""
pass

In [82]:
def find_glades(board):
    W, H = len(board), len(board[0])
    # TODO: do better
    def _is_glade(lx, ly, rx, ry):
        if lx == rx and ly == ry:
            return False
        for x in range(lx, rx+1):
            for y in range(ly, ry+1):
                if board[x][y] != 0:
                    return False
        return True
    
    for lx in range(W):
        for ly in range(H):
            for rx in range(lx, W):
                for ry in range(ly, H):
                    if _is_glade(lx, ly, rx, ry):
                        yield (lx, ly, rx, ry)


MAX_NUMS_PER_GLADE = 4
def make_board(w, h, seed):
    random.seed(seed)
    board = [[0 for _ in range(h)] for _ in range(w)]
    
    def _rec():
        if all(board[x][y] != 0 for x in range(w) for y in range(h)):
            return True
        # TODO: check for isolated cells, easy pruning
        glades = list(find_glades(board))      
        random.shuffle(glades)

        for lx, ly, rx, ry in glades:
          gw, gh = rx-lx+1, ry-ly+1
          assert gw*gh >= 2
          all_xy = [(x, y) for x in range(lx, rx+1) for y in range(ly, ry+1)]

          nns = list(reversed(range(2, min(MAX_NUMS_PER_GLADE, gw*gh)+1)))
          
          for nn in nns:
            nums = generate_sum_10(nn)
            for nxy in itertools.combinations(all_xy, nn):
                for n, (x, y) in zip(nums, nxy):
                    board[x][y] = n
                if _rec():
                    return True
                for x, y in nxy:
                    board[x][y] = 0
        return False
    
    if not _rec():
        return None
    return board
                


def generate_sum_10(n):
    seq = [random.randint(1, 9) for _ in range(n - 1)]
    if sum(seq) % 10 == 0:
        return generate_sum_10(n) # try again
    return seq + [10 - (sum(seq) % 10)]
    

board = make_board(5, 5, 35)
if board is not None:
    draw_board(board)
    compress_board(board)



4 6 7 8 1
3 8 4 1 9
5 7 2 1 8
3 5 5 9 2
9 5 8 2 8
Board{W: 5, H: 5, b: []int{4,6,7,8,1,3,8,4,1,9,5,7,2,1,8,3,5,5,9,2,9,5,8,2,8}}
